In [ ]:
!pip install sktime
!pip install skforecast
!pip install pmdarima

import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from dateutil.parser import parse
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import seaborn as sns
plt.rcParams.update({'figure.figsize': (10, 7), 'figure.dpi': 120}) # set figure sizes
import matplotlib.ticker as mtick
plt.style.use('fivethirtyeight')

# Creating data frames

# Fixmystreet.com data
df_fms = pd.read_csv('fms_ft_local.csv',
                 index_col='date', parse_dates=['date'], dayfirst=True)

# UK government data - all dat
df_nat = pd.read_csv('gov_ft_national.csv',
         index_col='date', parse_dates=['date'], dayfirst=True)

df_local = pd.read_csv('gov_ft_local.csv',
         index_col='date', parse_dates=['date'], dayfirst=True)

### Data Prep & CLeansing ###

# Check data types and null values - confirm all OK

df_fms.info()
df_nat.info()
df_local.info ()

### Visualise the data ###

# Defining a function that when called will output a line plot for us.
# We will pass the dataframe, the x and y columns and a title

def plot_df(df, x, y, title="", xlabel='Date', ylabel='Reports', dpi=100):
    """
    Good documentation
    """
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

# Call the function we just created to plot the line graph of the values
plot_df(df_fms, df_fms.index, y=df_fms.reports, title='Monthly Reports of Flytipping in Birmingham, UK (FMS')

### Trend in data - flytipping incidents have increased over time.
### It is harder to discern if there is a seasonal trend - there are peaks and troughs but no consistent pattern

# Extract the year and month from the index tp examine monthly and yearly trends separately

df_fms['year'] = df_fms.index.year
df_fms['month'] = df_fms.index.month

years = df_fms['year'].unique()

# Creating plots to break down trends (in years) and seasonality (in months)

fig, axes = plt.subplots(1, 2, figsize=(20,7), dpi= 80)    # subplots - creating two charts (one for year, one for month)

sns.boxplot(x='year', y='reports', data=df_fms, ax=axes[0])

sns.boxplot(x='month', y='reports', data=df_fms)

# Rotate the x-axis tick labels for better readability
axes[0].tick_params(axis='x', labelrotation=45)

# Set Title
axes[0].set_title('Year-wise Box Plot\n(The Trend)', fontsize=18);
axes[1].set_title('Month-wise Box Plot\n(The Seasonality)', fontsize=18)
plt.show

### Creating decompositions to see breakdown of trends and seasonality ###

from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse

# Additive Decomposition ( components are added together)

# result_add. will hold the components
result_add = seasonal_decompose(df_fms['reports'], model='additive', extrapolate_trend='freq')

# Plot
result_add.plot()
plt.show()

# Visualising against the National data for comparison, calling function already created to plot a line graph

plot_df(df_nat, df_nat.index, y=df_nat.reports, title='Yearly Reports of Flytipping in UK')

# Concentrating on 2015 - 2020 for a comparison with fixmystreet.com data which is dated 2015-2020

df_nat_1520 = df.loc['20150401':'20200401'] # subset of National ddata
plot_df(df_nat_1520, df_nat_1520.index, y=df_nat_1520.value, title='Yearly Fly-Tipping Reports (National gov.uk)')

### Using same decomposition method on local and national government data shows the same upwards trend in 2020. ###

# National data decomposition using technique from first dataset

from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse

result_add = seasonal_decompose(df_nat_1520['reports'], model='additive', extrapolate_trend='freq')

# Plot
result_add.plot()
plt.show()

# Local data decomposition as above

from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse

result_add = seasonal_decompose(df_local_1520['reports'], model='additive', extrapolate_trend='freq')

# Plot
result_add.plot()
plt.show()

### Building a predictive model using fixmystreet.com data ###

# Extract the year and month from the index

df_fms['year'] = df_fms.index.year
df_fms['month'] = df_fms.index.month

years = df_fms['year'].unique()

# Data preparation (index has been created above)
# ==============================================================================
df_fms['date'] = pd.to_datetime(df_fms.index, format='%Y-%m-%d')
df_fms = df_fms.set_index('date')
df_fms = df_fms.asfreq('MS')   # month start
df_fms = df_fms.sort_index()
df_fms.head()

# Checking for missing data
print(f'Number of rows with missing values: {data.isnull().any(axis=1).mean()}')

# Split data into train-test
# We are going to train on the first months and test on the last 24 months

steps = 24 # predicting the final 24 months
data_train = df_fms[:-steps]  

data_test  = df_fms[-steps:]   # this sets data_test to be the last 24 values

print(f"Train dates : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

# Plotting the test/train data
fig, ax = plt.subplots(figsize=(7, 2.5))
data_train['reports'].plot(ax=ax, label='train')
data_test['reports'].plot(ax=ax, label='test')
ax.legend();

# Manual inspection of parameters

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Plotting the Autocorrelation Function (ACF)
plt.figure(figsize=(14, 7))
plt.subplot(121)
plot_acf(data['reports'], ax=plt.gca(), lags=30, title='Autocorrelation Function')

# Plotting the Partial Autocorrelation Function (PACF)
plt.subplot(122)
plot_pacf(data['reports'], ax=plt.gca(), lags=30, title='Partial Autocorrelation Function')

plt.tight_layout()
plt.show()

from statsmodels.tsa.stattools import adfuller

#perform augmented Dickey-Fuller test
adfuller(df_reconstructed['resid'] 

# the p-value is 6.162750431878253e-07, so reject NULL, it is stationary

